### 존재하는 한글 이외의 문자 확인
- 원본데이터의 document에서 한글, 영어 제외해보기

In [3]:
import pandas as pd
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#train_df.drop("AI_id",axis=1,inplace=True)

test_df = pd.read_csv("2. 모델개발용자료.csv")
test_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#test_df.drop("AI_id",axis=1,inplace=True)

In [4]:
# train_df와 test_df 결합
set_df = pd.concat([train_df,test_df],axis=0).reset_index(drop=True)
set_df.shape

(1100000, 7)

In [8]:
import re
def simple_cleansing(df,col_lst):
    for col in col_lst:
        # null값 채우기
        df[col] = df[col].fillna("")
        
        # 양쪽 공백 삭제
        df[col] = df[col].apply(lambda x : x.strip())
        
        # 다중공백 -> 단일공백
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
        
        # 한글 자음모음 삭제
        df[col] = df[col].apply(lambda x : re.sub('([ㄱ-ㅎㅏ-ㅣ]+)',"",x))
    return df

In [9]:
set_df_clean = simple_cleansing(set_df.copy(),["text_obj","text_mthd","text_deal"])
set_df_clean["text_sum"] = set_df_clean["text_obj"] +" "+ set_df_clean["text_mthd"] +" "+ set_df_clean["text_deal"]

In [12]:
# 한글만 제외하는 함수
import re
def hangul_except(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    result = hangul.sub("",text)
    return result

# 영어만 제거하는 함수
def eng_except(text):
    eng = re.compile('[a-zA-Z]')
    result = eng.sub("",text)
    return result

# 특수기호 제거하는 함수


In [14]:
# 한글,영어 제외하고 숫자 및 특수기호만 남겨보기
df = set_df_clean
col = "text_sum"

df[col+"_nokr"] = df[col].apply(lambda x : hangul_except(x).strip()) # 한글이 없는 열 + 양쪽 공백제거
df[col+"_nokr"] = df[col+"_nokr"].apply(lambda x : eng_except(x).strip()) # 영어가 없는 열 + 양쪽 공백제거

In [21]:
# set_df_clean_nokr : 한글이 아닌 문자가 존재하는 데이터프레임
set_df_clean_nokr = set_df_clean[set_df_clean["text_sum_nokr"] != ""]
set_df_clean_nokr

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
1,id_0000002,G,47.0,472.0,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,.
5,id_0000006,C,29.0,291.0,철,절삭.용접,카프라배관자재,철 절삭.용접 카프라배관자재,.
6,id_0000007,I,56.0,561.0,음식점에서,접객시설을 갖추고,참치회(일본식),음식점에서 접객시설을 갖추고 참치회(일본식),()
10,id_0000011,C,10.0,102.0,멸치,"입고, 가공",,"멸치 입고, 가공",","
17,id_0000018,H,52.0,529.0,"선박,화물",운송화물,검수,"선박,화물 운송화물 검수",","
...,...,...,...,...,...,...,...,...,...
1099985,id_099986,NaN,NaN,NaN,철,"구입, 기계절단",,"철 구입, 기계절단",","
1099986,id_099987,NaN,NaN,NaN,영업장에서,공사수주,지붕판금공사.조립공사,영업장에서 공사수주 지붕판금공사.조립공사,.
1099990,id_099991,NaN,NaN,NaN,미용업,고객의 요구에 따라,"파마,염색,커트","미용업 고객의 요구에 따라 파마,염색,커트",",,"
1099992,id_099993,NaN,NaN,NaN,영업장에서,일반소비자에게,"떡볶이, 김밥 판매","영업장에서 일반소비자에게 떡볶이, 김밥 판매",","


In [55]:
res = ""
for i in set_df_clean_nokr["text_sum_nokr"]:
    res+=i
    
res_lst = [ ]
for i in range(len(res)):
    res_lst.append(res[i])

# 숫자 및 기호 리스트
res_lst = list(set(res_lst))

res = ""
for i in res_lst:
    res+=i
res_lst = list(res)

In [56]:
len(res_lst) # 숫자 및 특수기호 총 길이

85

In [61]:
# 숫자만 추출하기
import re

numbers = re.sub(r'[^0-9]', '', res)
numbers_lst = list(numbers)
len(numbers) # 숫자 총 길이

10

In [62]:
# 특수기호만 추출하기
import re
special = re.compile(r'[^ A-Za-z0-9가-힣+]')

# 특수문자만 찾기
result = special.findall(res)
codes = ''.join(result)
codes_lst = list(codes)
len(codes) # 특수문자 길이

73

In [161]:
# 문자열끼리 빼기 : 전체 - 특수문자
res_lst_del_codes = [x for x in res_lst if x not in codes_lst]
print(len(res_lst_del_codes))
res_lst_del_codes

12


['0', '1', '6', '2', '3', '7', '5', '+', '4', ' ', '8', '9']

In [509]:
# 문자열끼리 빼기 : 전체 - 숫자
res_lst_del_numbers = [x for x in res_lst if x not in numbers_lst]
len(res_lst_del_numbers)

75

In [557]:
" ".join(res_lst_del_numbers)

# 여기에 있는 영어는 특수기호 취급되는 영어들임 -> 다 원래 영어로 변환시키기

'５ ∼ 茶 . > ? ） ㎡ ㎣ Ｖ Ｇ Ⅱ ／ ² \\ ． 地 " ℃ ； ] - , ～ ㎠ 內 ＇ 外 ㎜ _ + 無 ㎟ ６ ３ （ @ } １ \' < ㎥ → ₂ ! Ⅲ = ， & ↘ / · ) ＆ # ？ ㆍ ;   金 ( 銅 : ^ ` % * ㎢ [ Ｐ Ｔ Ｌ º ~ ①'

In [554]:
num_lst = ['0', '1', '6', '2', '3', '7', '5', '+', '4', '8', '9']
# 특수문자를 제외한 삭제할 숫자 리스트

replace_lst = ["㎠","㎣","㎡","m²","㎥","㎢","㎟","²"] 
# 여기에 없는 특수기호는 공백으로 대체하기

no_except_lst = ["!","~","？","?"]
# 삭제 안할 리스트

except_lst = ["無","地","金","銅","茶","外","內","\'",";","；","℃"] # 한자는 삭제하는 게 좋을 듯

eng_lst = ["Ｖ","Ｇ","Ｐ","Ｌ","Ｔ"]
# 원래 영어로 변경

In [623]:
# 공백으로 처리해야할 특수문자 리스트 : del_special_lst
del_special_lst = ['５', '∼', '>', '）', 'Ⅱ', '／', '．', '"', ']', '-', ',', '～',
       '＇', '㎜', '_', '６', '３', '（', '@', '}', '１', '<', '→', '₂', 'Ⅲ',
       '=', '，', '&', '↘', '/', '·', '＆', '#', 'ㆍ', ' ', ':', '^', '`',
       '%', 'º', '①',"'"]

np.save("del_special_lst.npy",del_special_lst)

In [514]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("\?")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
1844,id_0001845,G,47.0,474.0,매장에서,일반인을대상으로,"?.,티셔츠","매장에서 일반인을대상으로 ?.,티셔츠","?.,"
2147,id_0002148,S,96.0,961.0,피부마사지?에서,일반고객대상으로,피부마사지관리,피부마사지?에서 일반고객대상으로 피부마사지관리,?
12895,id_0012896,C,26.0,261.0,"다이오드, 트랜지스터 및 유사 반도체소자,전자집적?",제조업,전자제품장비부품,"다이오드, 트랜지스터 및 유사 반도체소자,전자집적? 제조업 전자제품장비부품",", ,?"
14621,id_0014622,G,45.0,452.0,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",,,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",", ?"
14810,id_0014811,G,45.0,452.0,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",,,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",", ?"
...,...,...,...,...,...,...,...,...,...
1080786,id_080787,NaN,NaN,NaN,네일?에서,일반인대상으로,네일관리서비스,네일?에서 일반인대상으로 네일관리서비스,?
1088170,id_088171,NaN,NaN,NaN,영업장에서,고객의뢰를받아,도배? 실내내장식공사,영업장에서 고객의뢰를받아 도배? 실내내장식공사,?
1088656,id_088657,NaN,NaN,NaN,"전자상거래업, 천연비누제조, 천연화장품제조, 공방(?",,,"전자상거래업, 천연비누제조, 천연화장품제조, 공방(?",", , , (?"
1091825,id_091826,NaN,NaN,NaN,마사지?에서,고객에게,마사지서바스,마사지?에서 고객에게 마사지서바스,?


In [508]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("/")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
354,id_0000355,L,68.0,682.0,사업장에서,의뢰요청에 따라,부동산 관리/임대업,사업장에서 의뢰요청에 따라 부동산 관리/임대업,/
728,id_0000729,J,58.0,582.0,사업장에서,방과후교육콘텐츠,국책사업으로 교육s/w 개발공급,사업장에서 방과후교육콘텐츠 국책사업으로 교육s/w 개발공급,/
1302,id_0001303,C,22.0,222.0,"ABS/PC-ABS, PP","자재구입,입고,투입, 사출성형",,"ABS/PC-ABS, PP 자재구입,입고,투입, 사출성형","/-, ,,,"
1791,id_0001792,J,62.0,620.0,사업장에서,주문 받아,s/w개발,사업장에서 주문 받아 s/w개발,/
4005,id_0004006,J,62.0,620.0,고객의 요청,사업장에서,시스템 장애복구 A/S,고객의 요청 사업장에서 시스템 장애복구 A/S,/
...,...,...,...,...,...,...,...,...,...
1097930,id_097931,NaN,NaN,NaN,부품을 수입하여,산업사용자를 대상으로,풍력발전기 A/S,부품을 수입하여 산업사용자를 대상으로 풍력발전기 A/S,/
1099181,id_099182,NaN,NaN,NaN,사업장에서,IT산업관련,S/W개발,사업장에서 IT산업관련 S/W개발,/
1099247,id_099248,NaN,NaN,NaN,"잉크, OPP","인쇄, D/L",,"잉크, OPP 인쇄, D/L",", , /"
1099710,id_099711,NaN,NaN,NaN,건물내 설치된 서설물을,보수. 수리,보일러A/S,건물내 설치된 서설물을 보수. 수리 보일러A/S,. /


In [319]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("\+")]
# 삭제해도 됨 +

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
37538,id_0037539,G,47.0,472.0,소매업,일반인대상+,"과메기, 기타수산물","소매업 일반인대상+ 과메기, 기타수산물","+ ,"
39863,id_0039864,C,22.0,222.0,"PPT(폴리프로필렌+타르크), PA6(폴리프로필렌+아셀타린)","입고, 협력사부품입고",,"PPT(폴리프로필렌+타르크), PA6(폴리프로필렌+아셀타린) 입고, 협력사부품입고","(+), 6(+) ,"
50927,id_0050928,M,71.0,713.0,고객의 요청으로,영업장에서 -+광고문안작성 설계하여 광고물 제공,광고문안작성 설계하여 광고물 제공,고객의 요청으로 영업장에서 -+광고문안작성 설계하여 광고물 제공 광고문안작성 설계하...,-+
72598,id_0072599,I,56.0,561.0,음식점++에서,접객시설을 갖추고,육류구이,음식점++에서 접객시설을 갖추고 육류구이,++
83954,id_0083955,C,26.0,262.0,"P.G.C, P.C.B","PCB, PGC+실리콘",SOCKET,"P.G.C, P.C.B PCB, PGC+실리콘 SOCKET",".., .. , +"
108209,id_0108210,M,72.0,729.0,RF SENSOY,Data취득장비+진연장비통신결합,변전소주요장비이상유무진단,RF SENSOY Data취득장비+진연장비통신결합 변전소주요장비이상유무진단,+
125649,id_0125650,I,55.0,551.0,캠핑장에서,일반인을 대상으로+,캠핑장제공,캠핑장에서 일반인을 대상으로+ 캠핑장제공,+
137634,id_0137635,C,30.0,303.0,주물소재,"선삭1,2, 드릴+TAP",,"주물소재 선삭1,2, 드릴+TAP","1,2, +"
146242,id_0146243,C,20.0,204.0,Nafol20+A(알콜),"원료투입, 반응",계면활성제,"Nafol20+A(알콜) 원료투입, 반응 계면활성제","20+() ,"
151502,id_0151503,I,56.0,562.0,매장에서,일반인대상으로,"주류,안주류판매+","매장에서 일반인대상으로 주류,안주류판매+",",+"


In [304]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("\?")].to_csv("test.csv",encoding="utf-8-sig",
                                                                          index=False)

In [274]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("\+")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
37538,id_0037539,G,47.0,472.0,소매업,일반인대상+,"과메기, 기타수산물","소매업 일반인대상+ 과메기, 기타수산물","+ ,"
39863,id_0039864,C,22.0,222.0,"PPT(폴리프로필렌+타르크), PA6(폴리프로필렌+아셀타린)","입고, 협력사부품입고",,"PPT(폴리프로필렌+타르크), PA6(폴리프로필렌+아셀타린) 입고, 협력사부품입고","(+), 6(+) ,"
50927,id_0050928,M,71.0,713.0,고객의 요청으로,영업장에서 -+광고문안작성 설계하여 광고물 제공,광고문안작성 설계하여 광고물 제공,고객의 요청으로 영업장에서 -+광고문안작성 설계하여 광고물 제공 광고문안작성 설계하...,-+
72598,id_0072599,I,56.0,561.0,음식점++에서,접객시설을 갖추고,육류구이,음식점++에서 접객시설을 갖추고 육류구이,++
83954,id_0083955,C,26.0,262.0,"P.G.C, P.C.B","PCB, PGC+실리콘",SOCKET,"P.G.C, P.C.B PCB, PGC+실리콘 SOCKET",".., .. , +"
108209,id_0108210,M,72.0,729.0,RF SENSOY,Data취득장비+진연장비통신결합,변전소주요장비이상유무진단,RF SENSOY Data취득장비+진연장비통신결합 변전소주요장비이상유무진단,+
125649,id_0125650,I,55.0,551.0,캠핑장에서,일반인을 대상으로+,캠핑장제공,캠핑장에서 일반인을 대상으로+ 캠핑장제공,+
137634,id_0137635,C,30.0,303.0,주물소재,"선삭1,2, 드릴+TAP",,"주물소재 선삭1,2, 드릴+TAP","1,2, +"
146242,id_0146243,C,20.0,204.0,Nafol20+A(알콜),"원료투입, 반응",계면활성제,"Nafol20+A(알콜) 원료투입, 반응 계면활성제","20+() ,"
151502,id_0151503,I,56.0,562.0,매장에서,일반인대상으로,"주류,안주류판매+","매장에서 일반인대상으로 주류,안주류판매+",",+"


In [264]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("Ｖ")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
693277,id_0693278,G,47.0,473.0,사업장에서,일반인대상으로,ＴＶ수상기,사업장에서 일반인대상으로 ＴＶ수상기,ＴＶ


In [263]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("Ｔ")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
693277,id_0693278,G,47.0,473.0,사업장에서,일반인대상으로,ＴＶ수상기,사업장에서 일반인대상으로 ＴＶ수상기,ＴＶ


In [267]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("Ｐ")]
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("Ｇ")]
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("Ｌ")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
24414,id_0024415,G,47.0,477.0,사업장에서,일반소비자를 대상로,ＬＰＧ가스,사업장에서 일반소비자를 대상로 ＬＰＧ가스,ＬＰＧ
28879,id_0028880,G,47.0,477.0,LPG가스,소매,ＬＰＧ가스,LPG가스 소매 ＬＰＧ가스,ＬＰＧ
37138,id_0037139,G,47.0,477.0,소매업,일반인대상,ＬＰＧ가스,소매업 일반인대상 ＬＰＧ가스,ＬＰＧ
95673,id_0095674,G,47.0,477.0,사업장에서,일반인대상,ＬＰＧ가스.산소.질소.헬륨.알곤,사업장에서 일반인대상 ＬＰＧ가스.산소.질소.헬륨.알곤,ＬＰＧ....
189368,id_0189369,G,47.0,477.0,사업장에서,의뢰를 받아,ＬＰＧ가스 배달 소매,사업장에서 의뢰를 받아 ＬＰＧ가스 배달 소매,ＬＰＧ
276903,id_0276904,G,47.0,477.0,사업장에서,일반인을 대상으로,ＬＰＧ가스 가정집에 제공,사업장에서 일반인을 대상으로 ＬＰＧ가스 가정집에 제공,ＬＰＧ
329911,id_0329912,G,47.0,477.0,사업장에서,일반소비자에게,ＬＰＧ가스,사업장에서 일반소비자에게 ＬＰＧ가스,ＬＰＧ
330303,id_0330304,G,47.0,477.0,사업장에서,일반소비자에게,ＬＰＧ가스,사업장에서 일반소비자에게 ＬＰＧ가스,ＬＰＧ
330564,id_0330565,G,47.0,477.0,사업장에서,일반소비자에게,ＬＰＧ가스,사업장에서 일반소비자에게 ＬＰＧ가스,ＬＰＧ
331448,id_0331449,G,47.0,477.0,사업장에서,일반소비자에게,ＬＰＧ가스,사업장에서 일반소비자에게 ＬＰＧ가스,ＬＰＧ


In [257]:
test = "화물차를 가지고 고객의 주문을받아' 택배"
test.replace("\'","")

'화물차를 가지고 고객의 주문을받아 택배'

In [255]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("\'")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
6572,id_0006573,H,49.0,494.0,화물차를 가지고,고객의 주문을받아',택배,화물차를 가지고 고객의 주문을받아' 택배,'
9774,id_0009775,I,56.0,561.0,음식점에서,접객시설을갖추고',생선회등 판매,음식점에서 접객시설을갖추고' 생선회등 판매,'
12076,id_0012077,I,56.0,561.0,접객시설을 갖추고',매장에서,곱창및막창,접객시설을 갖추고' 매장에서 곱창및막창,'
15243,id_0015244,C,27.0,271.0,"트랜지스터,보드판","납땜, ASS'Y 검사",저주파치료기 제조,"트랜지스터,보드판 납땜, ASS'Y 검사 저주파치료기 제조",", , '"
18880,id_0018881,C,25.0,259.0,"다이케스팅, PCB","조립, Ass'y조립",,"다이케스팅, PCB 조립, Ass'y조립",", , '"
...,...,...,...,...,...,...,...,...,...
1075919,id_075920,NaN,NaN,NaN,부동산에서',부동산계약및 중개,부동산거래서비스,부동산에서' 부동산계약및 중개 부동산거래서비스,'
1076972,id_076973,NaN,NaN,NaN,고객을대상으로',사업장에서,기타부동산개발및공급업,고객을대상으로' 사업장에서 기타부동산개발및공급업,'
1077633,id_077634,NaN,NaN,NaN,주점에서,접객요원및무도시설을갖추고',맥주양주판매,주점에서 접객요원및무도시설을갖추고' 맥주양주판매,'
1089463,id_089464,NaN,NaN,NaN,구내식다에서,사업체직원을 대상으로',한식제공,구내식다에서 사업체직원을 대상으로' 한식제공,'


In [241]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("㎢")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
698935,id_0698936,G,47.0,471.0,슈퍼에서(165㎢미만),일반인에게소매,음식료품,슈퍼에서(165㎢미만) 일반인에게소매 음식료품,(165㎢)


In [237]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("%")].head(3)

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
54272,id_0054273,C,13.0,132.0,100%폴리에스테르,"원사구매, 와인더연사작업",,"100%폴리에스테르 원사구매, 와인더연사작업","100% ,"
58744,id_0058745,C,13.0,139.0,"도전성원단, 스폰지","성형, 합지","도전성 TAPE(50%), 도전성 가스캡(50%)","도전성원단, 스폰지 성형, 합지 도전성 TAPE(50%), 도전성 가스캡(50%)",", , (50%), (50%)"
76764,id_0076765,C,22.0,221.0,"고무, 철","재료입고, 성형","방진고무 (60%), 고무호스 (40%)","고무, 철 재료입고, 성형 방진고무 (60%), 고무호스 (40%)",", , (60%), (40%)"


In [234]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("%")]["소분류"].value_counts().head(5)

291.0    10
303.0     7
292.0     5
222.0     4
259.0     4
Name: 소분류, dtype: int64

In [223]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("1:1")].tail(5)

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
746867,id_0746868,R,91.0,911.0,매장에서,고객대상,1:1피팅,매장에서 고객대상 1:1피팅,1:1
761565,id_0761566,Q,86.0,869.0,상담교육,1:1 개별,"치료,상담","상담교육 1:1 개별 치료,상담","1:1 ,"
770460,id_0770461,S,96.0,969.0,개인을 대상으로,고용노동부인증 사회적기업에서,"병원다인간병, 1:1간병, 산모모우미서비스","개인을 대상으로 고용노동부인증 사회적기업에서 병원다인간병, 1:1간병, 산모모우미서비스",", 1:1,"
1063985,id_063986,NaN,NaN,NaN,동화세상에듀코에서,초중고학생상대로,1:1방문학습코칭,동화세상에듀코에서 초중고학생상대로 1:1방문학습코칭,1:1
1080015,id_080016,NaN,NaN,NaN,자선단체,청소년들을,상담및1:1자매결연,자선단체 청소년들을 상담및1:1자매결연,1:1


In [218]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("&")].head(3)

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
115,id_0000116,G,47.0,478.0,향초 & 디퓨저,소매,,향초 & 디퓨저 소매,&
876,id_0000877,P,85.0,856.0,"아이들,",학원에서,"CIP,BI 디자인&기획교육","아이들, 학원에서 CIP,BI 디자인&기획교육",", , &"
12537,id_0012538,C,28.0,281.0,"트랜스포머(변압기), 반도체부품","부품구입, 조립","전기전자&정밀기기수리, 무정전 전원장치(UPS)","트랜스포머(변압기), 반도체부품 부품구입, 조립 전기전자&정밀기기수리, 무정전 전원...","(), , &, ()"


In [214]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("＆")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
322380,id_0322381,P,85.0,856.0,홀리오＆꼬시까에서,고객의 의뢰를 받아,가구 공방,홀리오＆꼬시까에서 고객의 의뢰를 받아 가구 공방,＆
597517,id_0597518,M,70.0,701.0,판금 및 NCT 장비,R＆D 및 기술개발,"기술지원 및 시제품 개발, 전기, 통신","판금 및 NCT 장비 R＆D 및 기술개발 기술지원 및 시제품 개발, 전기, 통신","＆ , ,"


In [198]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("!")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
441759,id_0441760,I,56.0,561.0,앗! 뽕이다 에서,접객시설을 갖추고,중식,앗! 뽕이다 에서 접객시설을 갖추고 중식,!
476848,id_0476849,I,56.0,561.0,음식점에서,접객시설을 갖추고!,"닭발, 오돌뼈","음식점에서 접객시설을 갖추고! 닭발, 오돌뼈","! ,"
811834,id_0811835,I,55.0,551.0,아!레지던스에서,비교적단기,숙식제공,아!레지던스에서 비교적단기 숙식제공,!


In [197]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("₂")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
318040,id_0318041,C,29.0,291.0,"Ticl4, Tio₂","제조, 코팅",광촉매필터,"Ticl4, Tio₂ 제조, 코팅 광촉매필터","4, ₂ ,"
565371,id_0565372,C,20.0,201.0,"O₂(산소), N₂(질소)","입고, 펌프",,"O₂(산소), N₂(질소) 입고, 펌프","₂(), ₂() ,"


In [196]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("㎥")]["소분류"].value_counts()

471.0    94
472.0     4
Name: 소분류, dtype: int64

In [193]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("/")]
# 문자 사이의 특수기호는 단순히 삭제만

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
354,id_0000355,L,68.0,682.0,사업장에서,의뢰요청에 따라,부동산 관리/임대업,사업장에서 의뢰요청에 따라 부동산 관리/임대업,/
728,id_0000729,J,58.0,582.0,사업장에서,방과후교육콘텐츠,국책사업으로 교육s/w 개발공급,사업장에서 방과후교육콘텐츠 국책사업으로 교육s/w 개발공급,/
1302,id_0001303,C,22.0,222.0,"ABS/PC-ABS, PP","자재구입,입고,투입, 사출성형",,"ABS/PC-ABS, PP 자재구입,입고,투입, 사출성형","/-, ,,,"
1791,id_0001792,J,62.0,620.0,사업장에서,주문 받아,s/w개발,사업장에서 주문 받아 s/w개발,/
4005,id_0004006,J,62.0,620.0,고객의 요청,사업장에서,시스템 장애복구 A/S,고객의 요청 사업장에서 시스템 장애복구 A/S,/
...,...,...,...,...,...,...,...,...,...
1097930,id_097931,NaN,NaN,NaN,부품을 수입하여,산업사용자를 대상으로,풍력발전기 A/S,부품을 수입하여 산업사용자를 대상으로 풍력발전기 A/S,/
1099181,id_099182,NaN,NaN,NaN,사업장에서,IT산업관련,S/W개발,사업장에서 IT산업관련 S/W개발,/
1099247,id_099248,NaN,NaN,NaN,"잉크, OPP","인쇄, D/L",,"잉크, OPP 인쇄, D/L",", , /"
1099710,id_099711,NaN,NaN,NaN,건물내 설치된 서설물을,보수. 수리,보일러A/S,건물내 설치된 서설물을 보수. 수리 보일러A/S,. /


In [163]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("m²")]["소분류"].value_counts()

471.0    78
472.0     2
478.0     1
466.0     1
Name: 소분류, dtype: int64

In [136]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("∼")]["소분류"].value_counts()

471.0    20
Name: 소분류, dtype: int64

In [132]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("@")]
# 삭제하는 게 좋을 것 같음

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
421902,id_0421903,L,68.0,682.0,양우@에서,아파트입주자를위하여,관리서비스,양우@에서 아파트입주자를위하여 관리서비스,@


In [131]:
set_df_clean_nokr[set_df_clean_nokr["소분류"] == 682.0]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
332,id_0000333,L,68.0,682.0,계약에의거 부동산중개,부동산에서,중개.수수료,계약에의거 부동산중개 부동산에서 중개.수수료,.
354,id_0000355,L,68.0,682.0,사업장에서,의뢰요청에 따라,부동산 관리/임대업,사업장에서 의뢰요청에 따라 부동산 관리/임대업,/
559,id_0000560,L,68.0,682.0,부동산,고객을 대상,"토지,건물 중개대행","부동산 고객을 대상 토지,건물 중개대행",","
595,id_0000596,L,68.0,682.0,영업장에서,중개대행서비스,"토지,건물 중개서비스","영업장에서 중개대행서비스 토지,건물 중개서비스",","
631,id_0000632,L,68.0,682.0,부동산,고객의뢰로,"중개,매매","부동산 고객의뢰로 중개,매매",","
...,...,...,...,...,...,...,...,...,...
997540,id_0997541,L,68.0,682.0,사무실에서,계약을 통해 중개서비스,"매매,전.월세","사무실에서 계약을 통해 중개서비스 매매,전.월세",",."
997832,id_0997833,L,68.0,682.0,부동산 중개소에서,부동산중개알선,"토지,건물","부동산 중개소에서 부동산중개알선 토지,건물",","
998039,id_0998040,L,68.0,682.0,부동산중개,부동산에서,"토지,건물등","부동산중개 부동산에서 토지,건물등",","
998938,id_0998939,L,68.0,682.0,사무실에서,의뢰받아,센터관리(사업시설),사무실에서 의뢰받아 센터관리(사업시설),()


In [114]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("外")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
1,id_0000002,G,47.0,472.0,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,.
5,id_0000006,C,29.0,291.0,철,절삭.용접,카프라배관자재,철 절삭.용접 카프라배관자재,.
6,id_0000007,I,56.0,561.0,음식점에서,접객시설을 갖추고,참치회(일본식),음식점에서 접객시설을 갖추고 참치회(일본식),()
10,id_0000011,C,10.0,102.0,멸치,"입고, 가공",,"멸치 입고, 가공",","
17,id_0000018,H,52.0,529.0,"선박,화물",운송화물,검수,"선박,화물 운송화물 검수",","
...,...,...,...,...,...,...,...,...,...
1099985,id_099986,NaN,NaN,NaN,철,"구입, 기계절단",,"철 구입, 기계절단",","
1099986,id_099987,NaN,NaN,NaN,영업장에서,공사수주,지붕판금공사.조립공사,영업장에서 공사수주 지붕판금공사.조립공사,.
1099990,id_099991,NaN,NaN,NaN,미용업,고객의 요구에 따라,"파마,염색,커트","미용업 고객의 요구에 따라 파마,염색,커트",",,"
1099992,id_099993,NaN,NaN,NaN,영업장에서,일반소비자에게,"떡볶이, 김밥 판매","영업장에서 일반소비자에게 떡볶이, 김밥 판매",","


In [112]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("內")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
1079245,id_079246,NaN,NaN,NaN,교회內 도서관에서,주민들에게,도서대여,교회內 도서관에서 주민들에게 도서대여,內


In [111]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("茶")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
1097294,id_097295,NaN,NaN,NaN,사업장에서,수입하여 도소매,"차(茶),차관련기구를","사업장에서 수입하여 도소매 차(茶),차관련기구를","(茶),"


In [108]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("①")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
6162,id_0006163,C,25.0,251.0,"와이어, 벨트","부품입고, 결합①","구조대, 완강기","와이어, 벨트 부품입고, 결합① 구조대, 완강기",", , ① ,"


In [540]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("℃")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
897392,id_0897393,C,20.0,201.0,사업장에서,참나무를 1200℃로가열하여,참숯,사업장에서 참나무를 1200℃로가열하여 참숯,1200℃
978766,id_0978767,C,20.0,204.0,"병풀추출물, 호호바씨오일","성분용해, 80℃가온",,"병풀추출물, 호호바씨오일 성분용해, 80℃가온",", , 80℃"


In [531]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("\?")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
1844,id_0001845,G,47.0,474.0,매장에서,일반인을대상으로,"?.,티셔츠","매장에서 일반인을대상으로 ?.,티셔츠","?.,"
2147,id_0002148,S,96.0,961.0,피부마사지?에서,일반고객대상으로,피부마사지관리,피부마사지?에서 일반고객대상으로 피부마사지관리,?
12895,id_0012896,C,26.0,261.0,"다이오드, 트랜지스터 및 유사 반도체소자,전자집적?",제조업,전자제품장비부품,"다이오드, 트랜지스터 및 유사 반도체소자,전자집적? 제조업 전자제품장비부품",", ,?"
14621,id_0014622,G,45.0,452.0,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",,,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",", ?"
14810,id_0014811,G,45.0,452.0,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",,,"기타 자동차신품 부품 및 내장품 판매업,자동차 중고?",", ?"
...,...,...,...,...,...,...,...,...,...
1080786,id_080787,NaN,NaN,NaN,네일?에서,일반인대상으로,네일관리서비스,네일?에서 일반인대상으로 네일관리서비스,?
1088170,id_088171,NaN,NaN,NaN,영업장에서,고객의뢰를받아,도배? 실내내장식공사,영업장에서 고객의뢰를받아 도배? 실내내장식공사,?
1088656,id_088657,NaN,NaN,NaN,"전자상거래업, 천연비누제조, 천연화장품제조, 공방(?",,,"전자상거래업, 천연비누제조, 천연화장품제조, 공방(?",", , , (?"
1091825,id_091826,NaN,NaN,NaN,마사지?에서,고객에게,마사지서바스,마사지?에서 고객에게 마사지서바스,?


In [535]:
set_df_clean_nokr[set_df_clean_nokr["text_sum"].str.contains("℃")]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal,text_sum,text_sum_nokr
897392,id_0897393,C,20.0,201.0,사업장에서,참나무를 1200℃로가열하여,참숯,사업장에서 참나무를 1200℃로가열하여 참숯,1200℃
978766,id_0978767,C,20.0,204.0,"병풀추출물, 호호바씨오일","성분용해, 80℃가온",,"병풀추출물, 호호바씨오일 성분용해, 80℃가온",", , 80℃"


# 새로운 전처리 방식 적용

In [692]:
num_lst = ['0', '1', '6', '2', '3', '7', '5', '+', '4', '8', '9']
# 특수문자를 제외한 삭제할 숫자 리스트

replace_lst = ["㎠","㎣","㎡","m²","㎥","㎢","㎟","²"] 
# '면적' 이라는 한국어 낱말로 대체하기

no_except_lst = ["!","~","？","?"]
# 삭제하지 않고 남겨두는 특수문자 리스트

except_lst = ["無","地","金","銅","茶","外","內","\'",";","；","℃"] # 한자는 삭제하는 게 좋을 듯
except_id = ["id_0421903","id_027769","id_0718354","id_0843426"] # train 데이터에서 삭제할 데이터 id

eng_lst = ["Ｖ","Ｇ","Ｐ","Ｌ","Ｔ"]
# 원래 영어로 변경 VGPLT

del_special_lst = np.load("del_special_lst.npy")
# 공백으로 처리할것들
del_special_lst = ['５', '∼', '>', '）', 'Ⅱ', '／', '．', '"', ']', '-', ',', '～',
       '＇', '㎜', '_', '６', '３', '（', '@', '}', '１', '<', '→', '₂', 'Ⅲ',
       '=', '，', '&', '↘', '/', '·', '＆', '#', 'ㆍ', ' ', ':', '^', '`',
       '%', 'º', '①',"'"]

fail = ['\\', '+', ')', '(', '*', '['] # 공백으로 처리할 것들2

데이터 불러오기

In [695]:
import pandas as pd
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#train_df.drop("AI_id",axis=1,inplace=True)

test_df = pd.read_csv("2. 모델개발용자료.csv")
test_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#test_df.drop("AI_id",axis=1,inplace=True)

데이터 결합해서 set_df 생성

In [696]:
# train_df와 test_df 결합
set_df = pd.concat([train_df,test_df],axis=0).reset_index(drop=True)
set_df.shape

(1100000, 7)

새로운 클렌징 함수 new_data_cleansing_text

In [698]:
import re
from tqdm import tqdm 
tqdm.pandas()
def new_data_cleansing_text(df,col_lst):
    for col in col_lst:
        #print(col)
        # null값 채우기
        df[col] = df[col].fillna("")
        
        # 양쪽 공백 삭제
        #print("공백삭제")
        df[col] = df[col].apply(lambda x : x.strip())
        
        # 영어 대문자 -> 소문자
        df[col] = df[col].apply(lambda x : x.lower())
        
        # 특정 특수문자 삭제
        #print("특정삭제")
        except_lst = ["無","地","金","銅","茶","外","內","\'",";","℃"]
        for ex in except_lst:
            df[col] = df[col].apply(lambda x : re.sub(ex,'', x))
            
        #영어 문자 사이의 특수기호 삭제
        #print("영어사이삭제")
        df[col] = df[col].str.replace(pat="(?<=[a-z])[/$](?=[a-z])",repl = "",regex=True)
        
        # 특수기호 영어 원래 영어로 변경
        #print("영어변경")
        eng_lst = ["Ｖ","Ｇ","Ｐ","Ｌ","Ｔ"]
        eng = ["V","G","P","L","T"]
        for i in range(len(eng_lst)):
            df[col] = df[col].apply(lambda x : re.sub(eng_lst[i],eng[i], x))
            
        # 영어 대문자 -> 소문자
        df[col] = df[col].apply(lambda x : x.lower())

        # 숫자 제거
        #print("숫자제거")
        df[col] = df[col].apply(lambda x : re.sub(r'[0-9]', '', x))
    
        # 면적 관련 기호 처리
        replace_lst = ["㎠","㎣","㎡","㎥","m²","㎢","㎟","²"]
        for ex in replace_lst:
            df[col] = df[col].apply(lambda x : re.sub(ex,'면적', x))
            
        del_special_lst = np.load("del_special_lst.npy")
        # 공백으로 처리할것들
        for ex in del_special_lst:
            df[col] = df[col].apply(lambda x : re.sub(ex,' ', x))

        fail = ['\\', '+', ')', '(', '*', '['] # 공백으로 처리할 것들2
        
        df[col] = df[col].apply(lambda x : re.sub(r"[+]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[*]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[\\]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[(]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[)]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[[]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[]]"," ",x))
        
        df[col] = df[col].apply(lambda x : x.replace("."," "))
        # 다중공백 -> 단일공백
        #print("공백감소")
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
        
        no_except_lst = ["!","~","？","?"] # 삭제 안하고 남겨둘 특수문자
        df[col] = df[col].apply(lambda x : re.sub('？', '?', x))
    
        # 한글 자음모음 삭제
        df[col] = df[col].apply(lambda x : re.sub('([ㄱ-ㅎㅏ-ㅣ]+)',"",x))
        
        #양쪽 공백삭제
        df[col] = df[col].apply(lambda x : x.strip())
    return df

In [ ]:
set_df_clean = new_data_cleansing_text(set_df.copy(),["text_obj","text_mthd","text_deal"])
set_df_clean["text_sum"] = set_df_clean["text_obj"] +" "+ set_df_clean["text_mthd"] +" "+ set_df_clean["text_deal"]

In [ ]:
def hangul_except(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    result = hangul.sub("",text)
    return result

def eng_except(text):
    eng = re.compile('[a-zA-Z]')
    result = eng.sub("",text)
    return result

df = set_df_clean.copy()
col_lst = ["text_sum"]

df[col+"_nokr"] = df[col].apply(lambda x : hangul_except(x).strip()) # 한글이 없는 열 + 양쪽 공백제거
df[col+"_nokr"] = df[col+"_nokr"].apply(lambda x : eng_except(x).strip()) # 영어가 없는 열 + 양쪽 공백제거

In [ ]:
df_nokr = df[df["text_sum_nokr"] != ""]
#df_nokr.to_csv("test.csv",encoding="utf-8-sig",index=False)
df_nokr

In [ ]:
text = df_nokr.loc[1098942,"text_sum"]

In [ ]:
text